In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/gdrive', force_remount= True)

Mounted at /gdrive


In [ ]:
!pip install music21

In [ ]:
!nvidia-smi

Thu Apr 28 06:39:16 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import glob, pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential

from keras.layers import Dense, Dropout, LSTM, CuDNNLSTM

from keras.utils import np_utils

In [ ]:
# MIDI 파일을 불러오는 함수
midi = converter.parse('/gdrive/My Drive/midi_songs/BlueStone_LastDungeon.mid')

# MIDI 파일 내의 notes(음정, 박자를 포함하는 정보)를 불러온다.
notes_to_parse = midi.flat.notes

#불러온 notes의 갯수
print(np.shape(notes_to_parse))

(682,)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
for e in notes_to_parse[:20]:
  print(e, e.offset)

<music21.chord.Chord C#4 E4 A4> 0.0
<music21.note.Note A> 0.0
<music21.note.Note A> 1.5
<music21.note.Note A> 2.5
<music21.note.Note A> 3.5
<music21.note.Note A> 4.0
<music21.note.Note A> 5.5
<music21.note.Note A> 6.5
<music21.note.Note A> 7.0
<music21.chord.Chord D4 F4 B-4> 8.0
<music21.note.Note A> 8.0
<music21.note.Note A> 9.5
<music21.note.Note A> 10.5
<music21.note.Note A> 11.5
<music21.note.Note A> 12.0
<music21.note.Note A> 13.5
<music21.note.Note A> 14.5
<music21.note.Note A> 15.0
<music21.chord.Chord E-4 F#4 B4> 16.0
<music21.note.Note A> 16.0


In [ ]:
#midi_songs에 있는 Note 정보만 뽑아서 저장할 리스트

notes = []

for i, file in enumerate(glob.glob("/gdrive/My Drive/midi_songs/*")):
  midi =  converter.parse(file)
  print('\r', 'Parsing file', i, " ",file, end='')

  #Midi file을 notes로 나누어 다루기 위한 변수
  notes_to_parse = None

  #try / except : try 수행 중 에러 발생 시 except 수행

  try :# file has instrument parts
    s2 = instrument.partitionByInstrument(midi)
    notes_to_parse = s2.parts[0].recurse()
  except: # file has notes in a flat structure
    notes_to_parse = midi.flat.notes

  
  for e in notes_to_parse:

    # Note 인 경우 높이(Pitch), Octave 로 저장
    if isinstance(e, note.Note):
      notes.append(str(e.pitch))

     # Chord 인 경우 각 Note의 음높이(Pitch)를 '.'으로 나누어 저장
    elif isinstance(e, chord.Chord):
      # ':'.join([0, 1, 2]) : [0, 1, 2] -> [0:1:2]
      # str(n) for n in e.normalOrder 
      #     => e.normalOrder 라는 배열 내의 모든 원소 n에 대해 str(n) 해준 새 배열을 만든다.
      #        ex) str(i) for i in [1, 2, 3] => ['1', '2', '3']
      notes.append('.'.join(str(n) for n in e.normalOrder))

 Parsing file 91   /gdrive/My Drive/midi_songs/ff1battp.mid

In [ ]:
# MIDI 파일 정보를 다루기 쉽게 바꿔준다
# n_vocab : 모델 출력의 가짓수를 정하기 위해 Note의 총 가짓수를 센다(Note의 종류)
# set() : 중복되는 원소는 한번만 쓴다 / ex) set("Hello") => {'e', 'H', 'l', 'o'}
n_vocab = (len(set(notes)))

print('Classes of notes : ', n_vocab, '\n')
print('notes : ', notes[:500])
print('length of notes : ', len(notes), '\n')

# pitchnames : notes 배열의 모든 가능한 Note / Chord 를 정렬해놓은 배열  

pitchnames = sorted(set(item for item in notes))

print('pitchnames : ', pitchnames)
print('length of pitchnames : ', len(pitchnames), '\n')

# create a dictionary to map pitches to integers
# 음높이(Pitch)를 정수에 매핑하는 dictionary 자료형 생성
# ex) dict = {'key': value} => dict['key'] = value
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

print('note_to_int : ', note_to_int)

Classes of notes :  358 

notes :  ['4.9.10', 'E2', '4.9.10', 'E3', '4.9.10', 'E2', '4.9.10', 'E3', '4.9.10', 'E2', '4.9.10', 'E3', '4.9.10', 'E2', '4.9.10', 'E3', 'E-6', 'E6', 'E-5', 'A5', 'E5', 'E-6', 'E6', 'F5', 'E5', 'A5', 'E-5', 'E-6', 'E6', 'E5', 'A5', 'E-5', 'E-6', 'E6', 'F#5', 'A5', 'E5', 'E-6', 'E6', 'E-5', 'A5', 'E5', 'E-6', 'E6', 'F5', 'E5', 'A5', 'E-5', 'E-6', 'E6', 'E5', 'A5', 'E-5', 'E-6', 'E6', 'F#5', 'A5', 'E5', '4.9.10', 'E2', '4.9.10', 'E3', '4.9.10', 'E2', '4.9.10', 'E3', '4.9.10', 'E2', '4.9.10', 'E3', '4.9.10', 'E2', '4.9.10', 'E3', 'B5', 'E2', 'C6', 'B5', 'B-5', 'B5', 'B-5', 'A5', 'E3', 'B-5', 'A5', 'G5', 'A5', 'G5', 'B5', 'E2', 'C6', 'B5', 'B-5', 'B5', 'B-5', 'A5', 'E3', 'B5', 'A5', 'G5', 'A5', 'G5', 'B5', 'E2', 'C6', 'B5', 'B-5', 'B5', 'B-5', 'A5', 'E3', 'B5', 'A5', 'G5', 'A5', 'G5', 'F#5', 'E2', 'G5', 'F#5', 'E5', 'F#5', 'E5', 'E-5', 'E3', 'E5', 'E-5', 'B4', 'E-5', 'B4', '4.9.10', 'E2', '4.9.10', 'E3', '4.9.10', 'E2', '4.9.10', 'E3', 'E2', 'G4', 'E3', 'E4', 'E2

In [ ]:
# LSTM 모델을 위한 Training Dataset 생성
# 곡을 배끼는 것이 아닌 작곡이기 때문에 Validation이 필요없다

seq_len = 100 # 시퀀스 길이

# Pitch를 정수로 바꾸어 LSTM 모델의 입출력으로 만들어준다

net_in = []
net_out = []

# LSTM 모델의 입출력을 만들기 위해 ( 전체 길이 - 시퀀스 길이(=100) ) 만큼 반복
# ex) 입력 : 출력 짝지어주기
#     [0 ~ 99] : [100] / [1 ~ 100] : [101] / ... / [전체 길이-100 ~ 전체 길이-1] : [전체 길이]
for i in range(0, len(notes) - seq_len):
  # LSTM 모델 입력과 출력을 만들어준다
  seq_in = notes[i:i + seq_len] # ex) [0:100] => [0 ~ 99]
  seq_out = notes[i + seq_len]  # ex) [100]
  # LSTM은 문자열이 아닌 숫자를 입출력으로 하므로 문자열을 정수로 바꿔야 한다
  net_in.append([note_to_int[char] for char in seq_in]) # 배열 안의 모든 원소에 대해 실행
  net_out.append(note_to_int[seq_out]) # 출력값 하나에 대해 실행
print(np.shape(net_in))
print(np.shape(net_out))

(57077, 100)
(57077,)


In [ ]:
# LSTM 모델 입출력에 맞게 Dataset 전처리

# 시퀀스 길이(100) 만큼을 빼고 반복했으므로 100개 적은 패턴이 생긴다
n_patterns = len(net_in)
print('n_patterns : ', n_patterns)

# reshape the input into a format compatible with LSTM layers
# LSTM 입력에 맞는 모양으로 바꿔준다 : (샘플 수, 시퀀스 길이, 자료의 차원)
net_in = np.reshape(net_in, (n_patterns, seq_len, 1))
print('shape of net_in : ', net_in.shape)

# 데이터 범위 정규화 : 0 ~ (n_vocab - 1) => 0 ~ 1
net_in = net_in / float(n_vocab)

# 분류이므로 출력을 One-hot Vector로 만들어주어야 한다.
net_out = np_utils.to_categorical(net_out)
print('shape of net_out : ', net_out.shape)


n_patterns :  57077
shape of net_in :  (57077, 100, 1)
shape of net_out :  (57077, 358)


In [ ]:
# 모델 구성

# 데이터의 Feature(특징) 수 or Dimension(차원)
data_dim = net_in.shape[2]

# GPU 환경 : CuDNNLSTM() / CPU 환경 : LSTM()
model = Sequential(name="Sample_LSTM")

# return_sequences : True : Many to Many / False : Many to One
# seq_len : 입력으로 넣을 시계열 데이터의 길이 / data_dim : 각 데이터의 차원

model.add(CuDNNLSTM(512, input_shape=(seq_len, data_dim), return_sequences=True))

# GPU / CUDA / CuDNN 이 없는 환경에선 CuDNNLSTM만 LSTM으로 바꾸어 쓰면 됩니다.
# model.add(LSTM(512, input_shape=(seq_len, data_dim), return_sequences=True))
model.add(Dropout(rate=0.3))

model.add(CuDNNLSTM(512, return_sequences=True))
model.add(Dropout(rate=0.3))

model.add(CuDNNLSTM(512))

model.add(Dense(256))
model.add(Dropout(rate=0.3))

model.add(Dense(n_vocab, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()


Model: "Sample_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 cu_dnnlstm (CuDNNLSTM)      (None, 100, 512)          1054720   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 100, 512)          2101248   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 cu_dnnlstm_2 (CuDNNLSTM)    (None, 512)               2101248   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               

In [ ]:
# 모델 학습
# 음악 작곡이기 때문에 Validation Set이 없다
# LSTM : 13시간 / CuDNN : 3시간 반
# loss 는 0.7 ~ 0.8 만 되어도 충분히 작곡 능력을 갖는다.
model.fit(net_in, net_out, epochs=30, batch_size=64)

Epoch 1/30
892/892 [==============================] - 177s 190ms/step - loss: 4.6371
Epoch 2/30
892/892 [==============================] - 170s 190ms/step - loss: 4.5022
Epoch 3/30
892/892 [==============================] - 170s 190ms/step - loss: 4.3550
Epoch 4/30
892/892 [==============================] - 170s 190ms/step - loss: 4.1787
Epoch 5/30
892/892 [==============================] - 170s 191ms/step - loss: 3.9328
Epoch 6/30
892/892 [==============================] - 170s 190ms/step - loss: 3.6045
Epoch 7/30
892/892 [==============================] - 170s 190ms/step - loss: 3.2509
Epoch 8/30
892/892 [==============================] - 170s 190ms/step - loss: 2.8873
Epoch 9/30
892/892 [==============================] - 170s 190ms/step - loss: 2.5426
Epoch 10/30
892/892 [==============================] - 170s 190ms/step - loss: 2.2345
Epoch 11/30
892/892 [==============================] - 169s 190ms/step - loss: 1.9585
Epoch 12/30
892/892 [==============================] - 169s 190

In [ ]:
# 작곡을 위해 LSTM 모델 입력을 다시 만든다

# 위와 동일하므로 주석 생략

net_in = []
output = []

for i in range(0, len(notes) - seq_len, 1):
  seq_in = notes[i:i + seq_len]
  seq_out = notes[i + seq_len]

  net_in.append([note_to_int[char] for char in seq_in])
  output.append(note_to_int[seq_out])
  
n_patterns = len(net_in)

In [ ]:
# LSTM 모델이 작곡을 시작하기 위해 시작점으로써 랜덤한 시퀀스를 골라야 한다

# pattern : Dataset의 입력 전체 시퀀스 중 랜덤하게 고른 시퀀스
start = np.random.randint(0, len(net_in)-1)
pattern = net_in[start]
print('Random Sequence : ', pattern)

# int_to_note: 정수를 다시 Note로 바꾸기 위한 dictionary 자료형
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
print('int_to_note : ', int_to_note)

Random Sequence :  [330, 330, 330, 296, 317, 317, 330, 330, 330, 330, 296, 317, 332, 317, 330, 332, 330, 330, 332, 330, 319, 332, 356, 330, 356, 286, 317, 356, 299, 336, 354, 299, 354, 354, 299, 354, 286, 356, 286, 299, 354, 297, 286, 354, 284, 284, 356, 284, 286, 284, 299, 356, 332, 332, 284, 319, 317, 356, 317, 332, 330, 319, 330, 332, 330, 330, 330, 317, 317, 330, 330, 330, 330, 298, 330, 319, 317, 332, 317, 330, 332, 330, 330, 332, 330, 319, 332, 356, 330, 356, 286, 317, 356, 299, 336, 354, 299, 354, 354, 299]
int_to_note :  {0: '0', 1: '0.1', 2: '0.1.5', 3: '0.1.6', 4: '0.2', 5: '0.2.3.7', 6: '0.2.4.7', 7: '0.2.5', 8: '0.2.6', 9: '0.2.7', 10: '0.3', 11: '0.3.5', 12: '0.3.5.8', 13: '0.3.6', 14: '0.3.6.8', 15: '0.3.6.9', 16: '0.3.7', 17: '0.4', 18: '0.4.5', 19: '0.4.6', 20: '0.4.7', 21: '0.5', 22: '0.5.6', 23: '0.6', 24: '1', 25: '1.2', 26: '1.2.4.6.8.10', 27: '1.2.6', 28: '1.2.6.8', 29: '1.3', 30: '1.3.5', 31: '1.3.5.8', 32: '1.3.6', 33: '1.3.7', 34: '1.3.8', 35: '1.4', 36: '1.4.6'

In [ ]:
# LSTM 모델이 만든 출력값을 저장하기 위한 빈 리스트

pred_out = []

# generate 500 notes
for i in range(0, 500):
  # 랜덤하게 고른 시퀀스를 LSTM 모델 입력에 맞게 바꿔준다
  pred_in = np.reshape(pattern, (1, len(pattern), 1))

  # 입력 범위 정규화 / 0 ~ (n_vocab -1) => 0 ~ 1
  pred_in = pred_in / float(n_vocab)

  # LSTM 모델 사용
  prediction = model.predict(pred_in, verbose=0)

  # 출력 중 값이 가장 큰 Index 선택
  index = np.argmax(prediction)

  # 정수 값을 Note 값으로 변경
  result = int_to_note[index]
  print('\r', 'Predicted ', i, " ",result, end='')
  
  # LSTM이 만든 Note를 하나씩 리스트에 담는다
  pred_out.append(result)
  
  # 다음 입력을 위해 입력에 새 값 추가 후 가장 과거 값 제거
  # ex) [0:99] -> [1:100] -> ... -> [n : n + 99]
  pattern.append(index)
  pattern = pattern[1:len(pattern)]

In [ ]:
print('length of pred_out : ', len(pred_out))
print('pred_out : ', pred_out)

length of pred_out :  500
pred_out :  ['G3', 'A5', 'G5', 'A5', 'B5', 'G3', 'B3', 'A5', 'G3', 'A3', 'A3', 'G5', 'A3', 'A5', 'A3', 'B5', 'G5', 'E5', 'E5', 'A3', 'D5', 'D3', 'G5', 'D3', 'E5', 'E3', 'D5', 'E3', 'E5', 'E3', 'E3', 'E3', 'D3', 'D3', 'E3', 'E3', 'E3', 'E3', 'B4', 'E3', 'D5', 'D3', 'E5', 'D3', 'E3', 'E5', 'E3', 'E3', 'E5', 'E3', 'D5', 'E5', 'G5', 'E3', 'G5', 'A5', 'D3', 'G5', 'B5', 'F#3', 'G3', 'B5', 'G3', 'G3', 'B5', 'G3', 'A5', 'G5', 'A5', 'B5', 'G3', 'B3', 'A5', 'G3', 'A3', 'A3', 'G5', 'A3', 'A5', 'A3', 'B5', 'G5', 'E5', 'E5', 'A3', 'D5', 'D3', 'G5', 'D3', 'E5', 'E3', 'D5', 'E3', 'E5', 'E3', 'E3', 'E3', 'D3', 'D3', 'E3', 'E3', 'E3', 'E3', 'B4', 'E3', 'D5', 'D3', 'E5', 'D3', 'E3', 'E5', 'E3', 'E3', 'E5', 'E3', 'D5', 'E5', 'G5', 'E3', 'G5', 'A5', 'D3', 'G5', 'B5', 'F#3', 'G3', 'B5', 'G3', 'G3', 'B5', 'G3', 'A5', 'G5', 'A5', 'B5', 'G3', 'B3', 'A5', 'G3', 'A3', 'A3', 'G5', 'A3', 'A5', 'A3', 'B5', 'G5', 'E5', 'E5', 'A3', 'D5', 'D3', 'G5', 'D3', 'E5', 'E3', 'D5', 'E3', 'E5', 'E3',

In [ ]:
# LSTM 모델이 예측한 값들로부터 MIDI 파일을 만들어준다

offset = 0 # 음(Note/Chord)을 언제 들려줄지 정하는 timing offset (박자 정보를 대신 함

# MIDI 파일 생성을 위한 빈 리스트
output_notes = []

# create note and chord objects based on the values generated by the model
# LSTM 모델 예측 값을 하나씩 처리
for pattern in pred_out:

    # pattern이 Chord 일 때
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.') # ['8.1'].split('.') => ['8', '1']
        notes = [] # Note 정보를 저장할 빈 리스트

        # notes_in_chord의 텍스트를 Note 정보로 바꿔준다
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note)) # Text => 정수 => Note
            new_note.storedInstrument = instrument.Piano() # 악기 정보 설정
            notes.append(new_note) # notes 리스트에 더해준다

        # Note => Chord
        new_chord = chord.Chord(notes)
        new_chord.offset = offset # 시간 정보 설정
        output_notes.append(new_chord)
    # pattern이 Note 일 때
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)

    # 반복마다 offset을 증가시킨다 (고정 박자)
    offset += 0.5

# Note/Chord => Stream => MIDI File
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/gdrive/My Drive/test_output.mid')

'/gdrive/My Drive/test_output.mid'

In [ ]:
# 모델은 h5 파일 형태로 저장됩니다
# 경로에 주의합시다

model.save('/gdrive/My Drive/Sample_LSTM.h5')
# 모델을 불러오기 위해 지워줍니다
del model
# 저장되어 있는 모델을 불러오기 위한 load_model 함수
from keras.models import load_model
# 'model' 에 해당 모델을 불러옵니다
model = load_model('/gdrive/My Drive/Sample_LSTM.h5')